Data Preprocessing || `Model Training` || Streamlit App

---
# Solar Up -- Solar Panel Object Detection and Segmentation

WBS Coding School || Data Science Bootcamp || Final Project

---
<br>

**Use Google's GPU** to run this script to substantially reduce training time.

(Next to "Connection" -> Click arrow -> "Change runtime type" -> choose a GPU, e.g. "T4 GPU")


### Table of Contents
- [1. Download Data](#download)
- [2. Model Training](#training)
- [3. Export Model Weights](#export)

#### Libraries



In [ ]:
# ultralytics is the package you need for YOLOv8.
%pip install ultralytics

In [ ]:
# Update gdown for full functionality.
%pip install --upgrade gdown

In [ ]:
import os
import shutil
import torch

import ultralytics
from ultralytics import YOLO
from ultralytics import settings

from google.colab import files, drive

___
<a id="download"></a>
## 1.&nbsp; Download Data

We'll download the satellite images and a config yaml file from a Google Drive data dump.

The yaml tells the YOLOv8 model where to find the data and what the classes are. In our case, there's only one class, solar panels.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download kasmi_solar_detection.yaml config file.
!gdown #insert your file id here

# Insert the respective file ID here. To get it,
# go on your Google Drive > right click on 'kasmi_solar_detection.yaml' > Share > Copy link.
# The link will look something like:
# https://drive.google.com/file/d/<your_file_ID_here>/

Next, download the satellite images from a data-dump Drive.

In [ ]:
# Download zipped training data folder.
!gdown #insert your file id here

In [ ]:
# Display current path on Google Colab Machine.
%pwd
# Current path is '/content'.

# YOLOv8 expects the data folder to be contained in a directory called 'content/datasets/'.
!mkdir /content/datasets/


In [ ]:
# Unzip training data to that directory.
!unzip kasmi_solar_detection.zip -d /content/datasets/kasmi_solar_detection

'/content'

Check hardware and directory structure. Make sure a GPU is being used for model training.

In [ ]:
ultralytics.checks()

Ultralytics YOLOv8.1.28 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.5/78.2 GB disk)


In [ ]:
# Name of the dataset.
dataset_name = "kasmi_solar"

___
<a id="training"></a>
## 2.&nbsp; Model Training

Here we'll load a pretrained YOLOv8 model (small, medium or large) and train it on the solar panel satellite data. Subsequently, we'll check its performance on the validation set.

In [ ]:
# Load a COCO-pretrained YOLOv8m model.
model_name = 'yolov8l'
model = YOLO(f'{model_name}.pt')

In [ ]:
# Train the model on our custom dataset.
config_file = f'{dataset_name}_detection.yaml'
epochs = 20
img_size = 400
experiment = f'{model_name}_{epochs}'
device = 0 if torch.cuda.is_available() else "cpu" # device agnostic code

results = model.train(data=config_file, epochs=epochs, imgsz=img_size, name=experiment, device=device)

In [ ]:
# Validate the model.
metrics = model.val(data=config_file, epochs=epochs, imgsz=img_size, name=experiment, device=device)  # no arguments needed, dataset and settings remembered

metrics.box.map    # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps   # a list contains map50-95(B) of each category

___
<a id="export"></a>
## 3.&nbsp; Export Model Weights

In [ ]:
# Load the custom trained model.
path_to_best = f"runs/detect/{experiment}/weights/best.pt" # .pt = PyTorch model
model = YOLO(path_to_best)

# Export the model. The default format is 'torchscript' (PyTorch).
model.export()

In [ ]:
# Download trained model.
files.download(f"runs/detect/{experiment}/weights/best.torchscript")

In [ ]:
# Specify the output path for the ZIP archive for the training output.
output_path = f"models/training_data_{experiment}"
print(output_path)

shutil.make_archive(output_path, 'zip', f"/content/runs/detect/{experiment}")

In [ ]:
# Download training data.
files.download(f"models/training_data_{experiment}.zip")

In [ ]:
# Specify the output path for the ZIP archive for the evaluation data.
output_path = f"models/evaluation_data_{experiment}"
print(output_path)

shutil.make_archive(output_path, 'zip', f"/content/runs/detect/{experiment}")

In [ ]:
# Download evaluation data.
files.download(f"models/evaluation_data_{experiment}.zip")